In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
df_train=pd.read_csv('/kaggle/input/playground-series-s4e2/train.csv')
df_test=pd.read_csv('/kaggle/input/playground-series-s4e2/test.csv')
df_train.head()

In [ ]:
uniquevals=df_train['NObeyesdad'].unique()

In [ ]:
df_train.isnull().sum()

In [ ]:
mapping={
    'Male':1,
    'Female':0
}
df_train['Gender']=df_train['Gender'].map(mapping)
df_test['Gender']=df_test['Gender'].map(mapping)
df_train.head()

In [ ]:
mapping={
    'yes':1,
    'no':0
}
df_train['family_history_with_overweight']=df_train['family_history_with_overweight'].map(mapping)
df_test['family_history_with_overweight']=df_test['family_history_with_overweight'].map(mapping)

In [ ]:
df_train['FAVC']=df_train['FAVC'].map(mapping)
df_test['FAVC']=df_test['FAVC'].map(mapping)
df_train.head()

In [ ]:
df_train['SMOKE']=df_train['SMOKE'].map(mapping)
df_test['SMOKE']=df_test['SMOKE'].map(mapping)
df_train['SCC']=df_train['SCC'].map(mapping)
df_test['SCC']=df_test['SCC'].map(mapping)
df_train.head()

In [ ]:
c='MTRANS'
df_train[c].value_counts()

In [ ]:
expectedoutput=[]
for column in ['CAEC','CALC','MTRANS','NObeyesdad']:
    if(column=='NObeyesdad'):
        one_hot_encoded=pd.get_dummies(df_train[column]).astype(int)
        df_train=pd.concat([df_train.drop(column,axis=1),one_hot_encoded],axis=1)
        expectedoutput=one_hot_encoded.columns
    else:
        one_hot_encoded=pd.get_dummies(df_train[column]).astype(int)
        one_hot_encoded.columns=[f'{column}_{class_name}' for class_name in one_hot_encoded.columns]
        L=one_hot_encoded.columns
        df_train=pd.concat([df_train.drop(column,axis=1),one_hot_encoded],axis=1)
        one_hot_encoded=pd.get_dummies(df_test[column]).astype(int)
        one_hot_encoded.columns=[f'{column}_{class_name}' for class_name in one_hot_encoded.columns]
        df_test=pd.concat([df_test.drop(column,axis=1),one_hot_encoded],axis=1)
        for i in L:
            if(i not in df_test.columns):
                df_test[i]=0
        for i in one_hot_encoded.columns:
            if(i not in df_train.columns):
                df_train[i]=0
        
df_train.head()

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
expectedoutput=expectedoutput.tolist()
expectedoutput.append('id')
X=df_train.drop(expectedoutput,axis=1)
expectedoutput.pop()
Y=df_train[expectedoutput]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=5)

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
df_test=df_test[df_train.drop(uniquevals,axis=1).columns.tolist()]
xgb_submission=pd.DataFrame({val:np.zeros((df_test.shape[0],)) for val in uniquevals})
xgb_test=pd.DataFrame({val:np.zeros((X_test.shape[0],)) for val in uniquevals})
df_test.head()

In [ ]:
import lightgbm as lgb
X_submission=df_test.drop('id',axis=1)
#creating a model to predict each label separately
for val in xgb_submission.columns:
    if(val=='id'):
        continue
    dtrain=lgb.Dataset(X_train, label=Y_train[val])
    dtest=lgb.Dataset(X_test, label=Y_test[val])
    dsubmission=lgb.Dataset(df_test.drop('id',axis=1)[X_train.columns.tolist()])
    params = {
        'boosting_type': 'goss',  #Gradient-based One-Side Sampling
        'objective': 'binary',
        'metric': 'binary_error',
        'num_leaves': 31,  # Maximum number of leaves in one tree
        'learning_rate': 0.05,  # Learning rate
        'feature_fraction': 0.9,  # Percentage of features to consider in each iteration
        'bagging_fraction': 0.8,  # Percentage of data to use in each iteration
        'verbose': -1 , # Verbosity
    }
    num=200
    bst = lgb.train(params, dtrain, num, valid_sets=[dtest])
    xgb_test[val]=bst.predict(X_test) #validation prediction
    xgb_submission[val]=bst.predict(X_submission) #submission prediction
acc=accuracy_score(Y_test.idxmax(axis=1),xgb_test.idxmax(axis=1)) #validation accuracy
print(acc)

In [ ]:
xgb_submission.head()

In [ ]:
predicted_labels=xgb_submission.idxmax(axis=1)
predicted_labels.head()

In [ ]:
df_submission = pd.DataFrame({'id': df_test['id'], 'NObeyesdad': predicted_labels})
df_submission.head()

In [ ]:
df_submission.to_csv('/kaggle/working/submission.csv',index=False)